In [1]:
using LinearAlgebra

# Power Method

## Initialization and Checks

In [ ]:
A0 = [
    12  3  5  7  2  9  4  1  11  6;
    2   15  3  7  6  5  8  9  1   10;
    4   1   16  8  7  5  9  3  12  2;
    3   6   9   14  4  11  13  7  10  15;
    5   7   6   4   18  3  2  9  1   13;
    11  8   7   5  12  17  3  2  6   14;
    1   2   3   4  5   6   19  8  11  10;
    9   10  11  12  13  14  15  16  17  18;
    6   5   3   4  1   2   7   8   19  20;
    8   4   3   12  9   1   6   11  10  7;
]
A0 = [
    2   3   4   5   6   7   8   9  10  11  12;
    3   4   5   6   7   8   9  10  11  12  13;
    4   5   6   7   8   9  10  11  12  13  14;
    5   6   7   8   9  10  11  12  13  14  15;
    6   7   8   9  10  11  12  13  14  15  16;
    7   8   9  10  11  12  13  14  15  16  17;
    8   9  10  11  12  13  14  15  16  17  18;
    9  10  11  12  13  14  15  16  17  18  19;
   10  11  12  13  14  15  16  17  18  19  20;
   11  12  13  14  15  16  17  18  19  20  21;
   12  13  14  15  16  17  18  19  20  21  22
]
A0 = [
    4  2  3  0  1  0  0  0;
    0  5  4  1  0  0  0  0;
    2  3  6  0  0  1  0  0;
    1  2  0  7  3  0  0  0;
    0  0  2  3  8  4  1  0;
    0  0  0  1  4  9  3  2;
    0  0  0  0  1  3  10  4;
    0  0  0  0  0  2  4  11
]


A0 = [
    5   4   2;
    0   3   -1;
    0   0   1
]
A0 = [
    4  2  3  1;
    2  5  1  0;
    3  1  6  2;
    1  0  2  7
]
# first group of data 2-10 for the 10*10 non-symmetric one
# second group of data 11 for the 11*11 symmetric one
# third group of data 12-18 for the 8*8 non-symmetric one
# fourth group of data 19-20 for the 3*3 non-symmetric one
# fifth group of data 21-23 for the 4*4 symmetric one

4×4 Matrix{Int64}:
 4  2  3  1
 2  5  1  0
 3  1  6  2
 1  0  2  7

In [46]:
U, s, V = svd(A0)

SVD{Float64, Float64, Matrix{Float64}, Vector{Float64}}
U factor:
4×4 Matrix{Float64}:
 -0.478084   0.316656  -0.0857035   0.814751
 -0.298051   0.574778   0.688943   -0.325812
 -0.638866   0.107342  -0.592426   -0.478914
 -0.523881  -0.746885   0.408709    0.025866
singular values:
4-element Vector{Float64}:
 10.35155916620537
  6.288591756455024
  3.8912967942680567
  1.4685522830715483
Vt factor:
4×4 Matrix{Float64}:
 -0.478084   -0.298051  -0.638866  -0.523881
  0.316656    0.574778   0.107342  -0.746885
 -0.0857035   0.688943  -0.592426   0.408709
  0.814751   -0.325812  -0.478914   0.025866

In [36]:
eigen(A0)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 1.0
 3.0
 5.0
vectors:
3×3 Matrix{Float64}:
 -0.666667  -0.894427  1.0
  0.333333   0.447214  0.0
  0.666667   0.0       0.0

## To calculate the dominant singular value and its corresponding singular vectors

- $\mathbf{v_i} = (A^T A)^k \mathbf{x_0}$

In [37]:
# A FUNCTION FOR DOMINANT SINGULAR VECTORS
function pow_singular_dom(A, n)
    At = transpose(A)
    B = At * A
    x0 = randn(n)
    x = x0 / norm(x0)

    v0 = x0
    v1 = x0
    itr = 0
    while (true)
        itr += 1
        x = B * x
        x = x / norm(x)
        v1 = x
        if (norm(v1-v0) < 0.00001)
            println("$itr iterations in total for approximation")
            return v1
        end
        v0 = x
    end
end


pow_singular_dom (generic function with 1 method)

In [42]:
v_1 = pow_singular_dom(A0, 3)
sigma_1 = sqrt((dot(transpose(A0) * A0 * v_1, v_1)) / (transpose(v_1) * (v_1)))
u_1 = (A0 * v_1) / sigma_1
println(sigma_1)
println(v_1)
println(u_1)

8 iterations in total for approximation
6.907667263696954
[-0.6993781201795549, -0.6703477192281854, -0.24800036358780164]
[-0.9662136217913416, -0.25522983762729495, -0.035902187253743445]


## To calculate the non-trivial singular values and their corresponding singular vectors

- $\sigma_i = \sqrt{\frac{B\mathbf{v_i} \cdot \mathbf{v_i}}{\mathbf{v_i} \cdot \mathbf{v_i}}}$

- $\mathbf{u_i} = \frac{A_0 \mathbf{v_i}}{\sigma_i}$

- $A_{i} = A_{i-1} - \sigma_i \mathbf{u_i} \mathbf{v_i}^T$

In [43]:
function print_results(v, u, sigma, i)
    print("Singular value #$i: ")
    println(sigma)

    print("v_$i: ")
    println(v)

    print("u_$i: ")
    println(u)

    println()
end


# A FUNCTION FOR ALL SINGULAR VECTORS
function pow_singular_all(A, n)
    i = 1
    A0 = A
    B = transpose(A) * A
    v_cur = pow_singular_dom(A, n)
    sigma_cur = sqrt((dot(B * v_cur, v_cur)) / (transpose(v_cur) * (v_cur)))
    u_cur = (A0 * v_cur) / sigma_cur

    while true
        print_results(v_cur, u_cur, sigma_cur, i)
        if (i == n)
            break
        end
        i += 1
        A = A - (sigma_cur * u_cur * transpose(v_cur))
        B = transpose(A) * A
        v_cur = pow_singular_dom(A, n)
        sigma_cur = sqrt((dot(B * v_cur, v_cur)) / (transpose(v_cur) * (v_cur)))
        u_cur = (A0 * v_cur) / sigma_cur
        if (sigma_cur <= 10^(-5))
            i-=1
            break
        end
    end
    println("Found $i non-trivial singular values with corresponding singular vectors")
end

pow_singular_all (generic function with 1 method)

In [44]:
pow_singular_all(A0, 3)

8 iterations in total for approximation
Singular value #1: 6.907667263699527
v_1: [0.6993771847818838, 0.6703491655281049, 0.24799909209982993]
u_1: [0.9662134140848825, 0.2552306498243594, 0.03590200318464811]

5 iterations in total for approximation
Singular value #2: 2.769348208816954
v_2: [-0.43691950355136144, 0.6755575856708158, -0.5939051236174431]
u_2: [-0.24199825077061196, 0.9462796596999202, -0.21445664424812624]

2 iterations in total for approximation
Singular value #3: 0.7841195523236313
v_3: [0.5656614719273344, -0.30700805318165797, -0.7653581870331106]
u_3: [0.08870939723118881, -0.198523263513285, -0.9760733357114691]

Found 3 non-trivial singular values with corresponding singular vectors


## To calculate the dominant eigenvalue and its corresponding eigenvector

In [ ]:
# A FUNCTION FOR DOMINANT EIGENVECTORS
function pow_eigen_dom(A, n)
    y0 = randn(n)
    y = y0

    p0 = y0
    p1 = y0
    itr = 0
    while (true)
        itr += 1
        y = A * y
        y = y / norm(y)
        p1 = y
        if (norm(p1-p0) < 0.00001)
            println("$itr iterations in total for approximation")
            return p1
        end
        p0 = y
    end
end


pow_eigen_dom (generic function with 1 method)

In [31]:
p_1 = pow_eigen_dom(A0, 10)
lambda_1 = (dot(A0 * p_1, p_1)) / (transpose(p_1) * (p_1))
println(lambda_1)
println(p_1)

11 iterations in total for approximation
79.12731125722365
[0.21488025892152865, 0.26501404741756346, 0.2464192348103044, 0.3601675027302592, 0.2647534845032151, 0.3043757784548056, 0.2741101307984652, 0.5332376866912981, 0.29386051451286493, 0.290824932423552]


# The QR Method

In [75]:
#Where A is symmetric
using LinearAlgebra
using Random

n = 10
B = rand(1:3, 8, 8)
A = transpose(B)*B
A



8×8 Matrix{Int64}:
 30  27  25  28  22  22  29  24
 27  34  31  32  24  27  31  28
 25  31  33  32  21  24  28  27
 28  32  32  34  22  25  31  30
 22  24  21  22  24  23  23  22
 22  27  24  25  23  25  25  23
 29  31  28  31  23  25  33  29
 24  28  27  30  22  23  29  32

In [73]:
A = [2 0 4 ; 0 -3 0 ; 4 0 -4]

3×3 Matrix{Int64}:
 2   0   4
 0  -3   0
 4   0  -4

In [83]:
using LinearAlgebra

elapsed_time = @elapsed begin
temp_A = A
AQ = I
i = 0
while istriu(temp_A) == false
    Q, R = qr(temp_A)
    AQ = AQ * Q
    temp_A = R*Q

    temp_A[abs.(temp_A) .< 10^-5] .= 0
    if norm(temp_A - Diagonal(temp_A)) < 10^-5
        break;
    end
    i += 1
end
end
print(i)
display(temp_A)
print(elapsed_time)


80

8×8 Matrix{Float64}:
 215.958   0.0     0.0      0.0      0.0     0.0       0.0       0.0
   0.0    10.2255  0.0      0.0      0.0     0.0       0.0       0.0
   0.0     0.0     7.65159  0.0      0.0     0.0       0.0       0.0
   0.0     0.0     0.0      6.50469  0.0     0.0       0.0       0.0
   0.0     0.0     0.0      0.0      2.8972  0.0       0.0       0.0
   0.0     0.0     0.0      0.0      0.0     0.909393  0.0       0.0
   0.0     0.0     0.0      0.0      0.0     0.0       0.818294  0.0
   0.0     0.0     0.0      0.0      0.0     0.0       0.0       0.0357195

0.000886

In [119]:
#Displaying eigenvalues

v  = diag(temp_A)
r = round.(v,digits = 5)

println("Eigenvalues of A: ")
println(r)

Eigenvalues of A: 
[6.0, 3.0, 3.0]


In [55]:
#The eigenvectors are the columns of AQ
# Hi this is Jojo Jojo thinks these should be singular values?

display(AQ)

8×8 Matrix{Float64}:
 0.37077    0.42729     0.460441   …  -0.565161    0.142886  -0.0472726
 0.398507   0.0937668  -0.0456594     -0.241166   -0.177208  -0.0890704
 0.305786  -0.513515   -0.480015      -0.462569   -0.074499  -0.252332
 0.475749   0.189918   -0.175191       0.375853   -0.631227   0.0611067
 0.320661  -0.572269    0.55009        0.318465    0.153157  -0.323675
 0.378731   0.362463   -0.260031   …   0.401401    0.514732  -0.295694
 0.257281  -0.160801   -0.28382       -0.0143107   0.492988   0.575678
 0.267256  -0.155358    0.272091       0.0667016  -0.112873   0.631523

## Shifting

In [ ]:
elapsed_time = @elapsed begin
temp_A = A
n = size(temp_A, 1)
AQ = I
i = 0;

while istriu(temp_A) == false
    i += 1
    phi = eigvals(temp_A[n-1:n, n-1:n])
    REF = temp_A[n,n]
    if abs(phi[1] - REF) <= abs(phi[2] - REF)
        shift = phi[1]
    else
        shift = phi[2]
    end
    SH = shift * I

    Q, R = qr(temp_A - SH)
    AQ = AQ * Q
    temp_A = R * Q + SH

    temp_A[abs.(temp_A) .< 10^-5] .= 0
    if norm(temp_A - Diagonal(temp_A)) < 10^-5
        break;
    end

end
end
print(i)
display(temp_A)
print(elapsed_time)

390

8×8 Matrix{Float64}:
 215.958  0.0        0.0       0.0        0.0     0.0     0.0      0.0
   0.0    0.0357195  0.0       0.0        0.0     0.0     0.0      0.0
   0.0    0.0        0.818294  0.0        0.0     0.0     0.0      0.0
   0.0    0.0        0.0       0.909393   0.0     0.0     0.0      0.0
   0.0    0.0        0.0       0.0       10.2255  0.0     0.0      0.0
   0.0    0.0        0.0       0.0        0.0     2.8972  0.0      0.0
   0.0    0.0        0.0       0.0        0.0     0.0     7.65159  0.0
   0.0    0.0        0.0       0.0        0.0     0.0     0.0      6.50469

0.139329

In [72]:
display(AQ)

8×8 Matrix{Float64}:
 0.306526  -0.45233     0.633406   …   0.277559    0.182164    0.208796
 0.381342   0.317289   -0.165544       0.272963    0.539108    0.070704
 0.346142  -0.593709   -0.137885       0.0671155  -0.326352   -0.0894484
 0.301764  -0.0929886   0.0402814     -0.753057    0.029325   -0.363893
 0.363696  -0.135222   -0.327679      -0.280368    0.441485    0.316355
 0.365445   0.0170834  -0.52742    …   0.332524   -0.39593    -0.0298661
 0.367398   0.453949    0.289733      -0.220455   -0.465958    0.535498
 0.385925   0.330064    0.285074       0.197671   -0.0051588  -0.650581

# Problem Shooting

In [ ]:
# Leading eigenvectors = those that correspond to singular values > 10^(-5)
    # create a limit after each calculation of singular value and if reach 10^(-5), disregard the rest of them

# Power method: k -> infinity? how to explain implementing this in julia
    # can calculate a reasonable based on source and cite it and put it in codes
    # or do an epsilon = x_k+1 - x_k and mtemp_Ae it less than 0.1
    # dominant eigenvalues that don't differ much --> put the checks in code and state limitations

# what kind of matrices is our algorithm supposed to work on well? Now power method works well with symmetric matrices; QR method only works on squares, well on symmetric, not that reliable on non-symmetric square matrices.
    # invertible ones
    # maybe because of complex eigenvalues the square ones wouldn't mtemp_Ae sense
    # can focus on symmetric ones for now for QR method

# efficiency of shifting: it gives the correct answers but we don't know about the efficiency
    # mtemp_Ae it as big as possible?
    # if can't mtemp_Ae distinguishable just state the possible reason in limitations

# explanation: we're explaining the steps of the methods and the improvements on code we could possibly mtemp_Ae -- do we need to explain the proofs of the methods or anything else? (look at project)
    # how methods work + proof of math behind it + explanation of code (and improvements)